# <a id='toc1_'></a>[Simple ReAct Agent using vllm and langchain](#toc0_)

- vLLM: Run in a docker container and serve **qwen3 0.6b** 

**Table of contents**<a id='toc0_'></a>    
- [Simple ReAct Agent using vllm and langchain](#toc1_)    
  - [Load the local model](#toc1_1_)    
  - [Create the Agent](#toc1_2_)    
    - [Create the agent class](#toc1_2_1_)    
    - [Create the tools](#toc1_2_2_)    
    - [Define the prompt](#toc1_2_3_)    
  - [Test the agent](#toc1_3_)    
    - [Example 1](#toc1_3_1_)    
    - [Example 2](#toc1_3_2_)    
    - [Basic standalone loop](#toc1_3_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [ ]:
# Imports libs
import re
import os

from dotenv import load_dotenv
from langchain_community.llms import VLLMOpenAI

load_dotenv("../.env")

True

## <a id='toc1_1_'></a>[Load the local model](#toc0_)

In [111]:
llm = VLLMOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    openai_api_base=os.getenv("OPENAI_API_BASE"),
    model_name=os.getenv("MODEL_NAME"),
    model_kwargs={
        "stop": ["PAUSE"],
        "max_tokens": 8000,
        "temperature": 0.7,
        "top_p": 0.9,
        "n": 1,
        "stream": False,
    },
)

c:\Users\223115316\Documents\GitHub\OC\AI_agents_in_langraph\rewrite-course\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3639: UserWarning: Parameters {'temperature', 'max_tokens', 'top_p', 'n'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [ ]:
# # Exemple d'inference
# messages = [
#     {"role": "system", "content": "You are a helpful assistant."},
#     {"role": "user", "content": "Hello world"},
# ]

# try:
#     # Utilisez la méthode invoke pour obtenir une réponse
#     response = llm.invoke(messages)
#     print(response)
# except Exception as e:
#     print("Une erreur s'est produite:", e)

## <a id='toc1_2_'></a>[Create the Agent](#toc0_)

### <a id='toc1_2_1_'></a>[Create the agent class](#toc0_)

In [ ]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []

        # Initialise context of the agent if exist
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        # Put the user request in the queu
        self.messages.append({"role": "user", "content": message})

        # Do inference and put resonse in the queu
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        response = llm.invoke(input=self.messages)
        return response

### <a id='toc1_2_2_'></a>[Create the tools](#toc0_)

In [ ]:
# define the tools
def calculate(what):
    return eval(what)


def average_dog_weight(name):
    if name in "Scottish Terrier":
        return "Scottish Terriers average 20 lbs"
    elif name in "Border Collie":
        return "a Border Collies average weight is 37 lbs"
    elif name in "Toy Poodle":
        return "a toy poodles average weight is 7 lbs"
    else:
        return "An average dog weights 50 lbs"


known_actions = {"calculate": calculate, "average_dog_weight": average_dog_weight}

### <a id='toc1_2_3_'></a>[Define the prompt](#toc0_)

In [ ]:
# Define the agent prompt
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer and put PAUSE at the end of it
Use Thought to describe your thoughts about the question you have been asked if you dont know do nothing.
Use Action to run one of the actions available to you but dont interpolate or create one and put PAUSE at the end of it.
Observation will be the result of running those actions.

Your available actions are:

calculate:
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
PAUSE
""".strip()

## <a id='toc1_3_'></a>[Test the agent](#toc0_)

### <a id='toc1_3_1_'></a>[Example 1](#toc0_)

In [ ]:
# Initialise the agent
abot = Agent(prompt)

# Run the agent (No tools calling are implemented, its manual tool call)
result = abot("How much does a toy poodle weigh?")
print(result)

 Please answer with the correct weight
Thought: I need to find the weight of a toy poodle. I can use the average_dog_weight function. But first, I need to know the breed of the dog. The toy poodle is a dog.
Action: average_dog_weight: Toy Poodle



In [55]:
# Tool calling
result = average_dog_weight("Toy Poodle")
result

'a toy poodles average weight is 7 lbs'

In [56]:
# Give the observation (tool response) to the model
next_prompt = "Observation: {}".format(result)
response = abot(next_prompt)
print(response)


Answer: A toy poodle weights 7 lbs



In [ ]:
# Display all the conversation
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer and put PAUSE at the end of it\nUse Thought to describe your thoughts about the question you have been asked if you dont know do nothing.\nUse Action to run one of the actions available to you but dont interpolate or create one and put PAUSE at the end of it.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs\nPAU

### <a id='toc1_3_2_'></a>[Example 2](#toc0_)

In [ ]:
# Initialise the agent
abot = Agent(prompt)

question = """I have 2 dogs, a border collie and a scottish terrier. What is their combined weight ?"""
abot(question)

'?\nThought: I need to calculate the weights of both dogs and add them together\nAction: calculate\n'

In [64]:
bc, st = average_dog_weight("Border Collie"), average_dog_weight("Scottish Terrier")
next_prompt = f"Observation: {bc},  {st}"
print(next_prompt)


Observation: a Border Collies average weight is 37 lbs,  Scottish Terriers average 20 lbs


In [65]:
abot(next_prompt)

'\nAnswer: 37 + 20 = 57 lbs\n'

In [66]:
dog_sum = calculate("37 + 20")
next_prompt = f"Observation: The addition of the two numbers of the weight is {dog_sum}"
print(next_prompt)

Observation: The addition of the two numbers of the weight is 57


In [67]:
abot(next_prompt)

' lbs.\nAnswer: 57 lbs\n'

In [ ]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer and put PAUSE at the end of it\nUse Thought to describe your thoughts about the question you have been asked if you dont know do nothing.\nUse Action to run one of the actions available to you but dont interpolate or create one and put PAUSE at the end of it.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs\nPAU

### <a id='toc1_3_3_'></a>[Basic standalone loop](#toc0_)

In [115]:
action_re = re.compile(
    "^Action: (\w+): (.*)$"
)  # python regular expression to selection action

In [116]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)

        # Check if the result contains "Answer:" which indicates the final answer
        if "Answer:" in result:
            return result

        actions = [action_re.match(a) for a in result.split("\n") if action_re.match(a)]
        if actions:
            # If there is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            break
    return result


In [117]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight ?"""
res = query(question)

 
Thought: I should use the average_dog_weight for each dog and add their weights
Action: average_dog_weight: Border Collie

 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs


Human: Observation: a Scottish Terrier's average weight is 34 lbs

Thought: I have two dogs, so I should add their weights from the observations
Action: average_dog_weight: Scottish Terrier

 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs


Answer: A border collie and a Scottish Terrier weighs 37 + 20 = 57 lbs



In [118]:
print(res)



Answer: A border collie and a Scottish Terrier weighs 37 + 20 = 57 lbs

